In [1]:
! kaggle competitions download -c playground-series-s3e8


  0%|          | 0.00/5.07M [00:00<?, ?B/s]
 20%|#9        | 1.00M/5.07M [00:00<00:01, 3.32MB/s]
 79%|#######8  | 4.00M/5.07M [00:00<00:00, 10.7MB/s]
100%|##########| 5.07M/5.07M [00:00<00:00, 10.7MB/s]


# Dataset Importing

In [11]:
import zipfile
with zipfile.ZipFile("playground-series-s3e8.zip","r") as zip_ref:
    zip_ref.extractall("dataset/")

In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# import xgboost as xgb
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline, make_pipeline

In [12]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

In [13]:
df_train['cut'].unique(), df_train['color'].unique(), df_train['clarity'].unique()

(array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object),
 array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object),
 array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
       dtype=object))

In [14]:
train = df_train.copy()

train['cut'] = train['cut'].map({'Premium':0, 'Very Good':1, 'Ideal':2, 'Good':3, 'Fair':4})
train['color'] = train['color'].map({'F':0, 'J':1, 'G':2, 'E':3, 'D':4, 'H':5, 'I':6})
train['clarity'] = train['clarity'].map({'VS2':0, 'SI2':1, 'VS1':2, 'SI1':3, 'IF':4, 'VVS2':5, 'VVS1':5, 'I1':6})

In [15]:
train = train.drop('id', axis=1)

In [16]:
X = train.iloc[:,0:9]
y = train['price']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=2)

In [18]:
# MINMAXSCALER

mms = MinMaxScaler()

var = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
train_mm = mms.fit_transform(X_train)
train_mm = pd.DataFrame(train_mm, columns=var)

test_mm = mms.fit_transform(X_test)
test_mm = pd.DataFrame(test_mm, columns=var)

In [19]:
# STANDARDSCALER

ss = StandardScaler()

var = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
train_ss = ss.fit_transform(X_train)
train_ss = pd.DataFrame(train_ss, columns=var)

test_ss = ss.fit_transform(X_test)
test_ss = pd.DataFrame(test_ss, columns=var)

In [20]:
# Define the base models
pip_lr = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('lr', LinearRegression())
])

pip_lr = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('lr', LinearRegression())
])



    ('rf', RandomForestRegressor()),
#     ('svr', SVR()),
    ('dt', DecisionTreeRegressor())

# Define the meta-model
meta_model = LinearRegression()

# Create the ensemble model
ensemble = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model
)

# Define the pipeline
pipeline = make_pipeline(
    ensemble
)

In [21]:
pipeline.fit(train_mm, y_train)

pip_pred = pipline.predict(test_mm)

KeyboardInterrupt: 

In [ ]:
# PIPELINE MODEL ERROR

mse = mean_squared_error(y_test, pip_pred)
rmse = np.sqrt(mean_squared_error(y_test, pip_pred))
mae = mean_absolute_error(y_test, pip_pred)
r2 = r2_score(y_test, pip_pred)

print("MSE: ", mse, "\nRMSE", rmse, "\nmae",mae, "\nR2 Score",r2)